In [ ]:
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable

# Relative import of code from distiller, w/o installing the package
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import distiller
import models
from apputils import *

## Performance overview



In [ ]:
model = models.create_model(pretrained=False, dataset='imagenet', arch='resnet18', parallel=False)

In [ ]:
dummy_input = Variable(torch.randn(1, 3, 224, 224), requires_grad=False)
df = distiller.model_performance_summary(model, dummy_input, batch_size=1)
display(df)

total_macs = df['MACs'].sum()

print("Total MACs: " + "{:,}".format(total_macs))

In [ ]:
df_compute = df['MACs']
ax = df_compute.plot.bar(figsize=[15,10], title="MACs");
ax.set_xticklabels(df.Name, rotation=90);

In [ ]:
df['FM volume'] = df['IFM volume'] + df['OFM volume']
df_footprint = df[['FM volume', 'Weights volume']]
ax = df_footprint.plot.bar(figsize=[15,10], title="Footprint");
ax.set_xticklabels(df.Name, rotation=90);

## ResNet20 channel pruning using SSL

Let's see how many MACs we saved by using SSL to prune filters from ResNet20:

In [ ]:
resnet20_dense = models.create_model(pretrained=False, dataset='cifar10', arch='resnet20_cifar', parallel=True)
resnet20_sparse = models.create_model(pretrained=False, dataset='cifar10', arch='resnet20_cifar', parallel=True)
checkpoint_file = "../examples/ssl/checkpoints/checkpoint_trained_channel_regularized_resnet20_finetuned.pth.tar" 
load_checkpoint(resnet20_sparse, checkpoint_file);

In [ ]:
dummy_input = Variable(torch.randn(1, 3, 32, 32), requires_grad=False)
df_dense = distiller.model_performance_summary(resnet20_dense, dummy_input, batch_size=1)
df_sparse = distiller.model_performance_summary(resnet20_sparse, dummy_input, batch_size=1)

dense_macs = df_dense['MACs'].sum()
sparse_macs = df_sparse['MACs'].sum()

print("Dense MACs: " + "{:,}".format(int(dense_macs)))
print("Sparse MACs: " + "{:,}".format(int(sparse_macs)))
print("Saved MACs: %.2f%%" % ((1 - sparse_macs/dense_macs)*100))